# Characteristic Time with Details

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from viz.style import set_style
from viz.printing import printer
from viz.layout import layout_fig
from sto_rheed.Dataset import RHEED_parameter_dataset
from sto_rheed.Viz import Viz
from sto_rheed.Analysis import analyze_curves, remove_outlier, smooth
from sto_rheed.Packed_functions import visualize_characteristic_time, violinplot_characteristic_time
from sto_rheed.Fit import NormalizeData

printing = printer(basepath = '../Figures/4.Characteristic_time/')
printing_plot = printer(basepath = '../Figures/4.Characteristic_time/', fileformats=['png', 'svg'])

set_style("printing")

seq_colors = ['#00429d','#2e59a8','#4771b2','#5d8abd','#73a2c6','#8abccf','#a5d5d8','#c5eddf','#ffffe0']
bgc1, bgc2 = colors.hex2color(seq_colors[0]), colors.hex2color(seq_colors[5])
color_blue = (44/255,123/255,182/255)
color_orange = (217/255,95/255,2/255)
color_purple = (117/255,112/255,179/255)

## 1. Examples of Fitted RHEED Intenstity Curve

In [ ]:
spot = 'spot_2'
metric = 'img_sum'
camera_freq = 500
fit_settings = {'savgol_window_order': (15, 3), 'pca_component': 10, 'I_diff': 15000, 
                'unify':False, 'bounds':[0.001, 1], 'p_init':[0.1, 0.4, 0.1]}

## 2. Analyze the Decay Curve

### 2.1 Sample 1 - treated_213nm

#### 2.1.1 Fitting Process

In [ ]:
path = 'D:/STO_STO-Data/RHEED/STO_STO_Berkeley/test6_gaussian_fit_parameters_all-04232023.h5'
D1_para = RHEED_parameter_dataset(path, camera_freq=500, sample_name='treated_213nm')

growth_list = ['growth_1', 'growth_2', 'growth_3', 'growth_4', 'growth_5']
D1_para.viz_RHEED_parameter_trend(growth_list, spot='spot_2', metric_list=['img_sum'], head_tail=(100,300), interval=200)

In [ ]:
growth_dict = {'growth_1':1, 'growth_2':1, 'growth_3':1, 'growth_4':3, 'growth_5':3}
x_all, y_all = D1_para.load_multiple_curves(growth_dict.keys(), spot, metric, x_start=0, interval=0)

parameters_all, x_coor_all, info = analyze_curves(D1_para, growth_dict, spot, metric, interval=0, fit_settings=fit_settings)
[xs_all, ys_all, ys_fit_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels_all, losses_all] = info

# define two color regime
x_y1 = x_coor_all[losses_all[:,0]>losses_all[:,1]]
x_y2 = x_coor_all[losses_all[:,0]<losses_all[:,1]]
color_array = Viz.two_color_array(x_coor_all, x_y1, x_y2, bgc1, bgc2, transparency=0.5)
color_array = np.concatenate([np.expand_dims(x_coor_all, 1), color_array], axis=1)
Viz.plot_fit_details(xs_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels=range(len(x_all)), 
                     save_name='S10-fitting_details_treated_213nm', printing=printing_plot)

#### 2.1.2 Fitted parameters

In [ ]:
fig, axes = layout_fig(4, 1, figsize=(6.5, 2*4))
Viz.plot_curve(axes[0], x_all, y_all, plot_type='lineplot', xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 130), yaxis_style='sci')
Viz.plot_curve(axes[1], x_coor_all, parameters_all[:,0], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted a (a.u.)', xlim=(-2, 130))
Viz.plot_curve(axes[2], x_coor_all, parameters_all[:,1], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted b (a.u.)', xlim=(-2, 130))
Viz.plot_curve(axes[3], x_coor_all, parameters_all[:,2], plot_type='lineplot', xlabel='Time (s)', ylabel='Characteristic Time (s)', xlim=(-2, 130))
printing_plot.savefig(fig, 'S7-treated_213nm-a_b_tau')
plt.show()

#### 2.1.3 Remove Outliers

Because there is hard boundary when characteristic >= 1, so we can assume the characteristic data that >= 0.95 are outliers and remove from plot. And we also process the data with less noise to observe the trend.

In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 5)

x_coor_all_clean_sample1 = np.copy(x_coor_all_clean)
tau_clean_sample1 = np.copy(tau_clean)
tau_smooth_sample1 = np.copy(tau_smooth)

fig, ax = plt.subplots(1, 1, figsize=(6, 2), layout='compressed')
Viz.plot_curve(ax, x_coor_all_clean_sample1, tau_clean_sample1, curve_y_fit=tau_smooth_sample1, plot_type='lineplot', 
            plot_colors=['k', '#bc5090'], xlabel='Time (s)', ylabel='Characteristic Time (s)', 
            yaxis_style='linear', markersize=3, xlim=(-2, 130))
plt.show()

#### 2.1.4 Plot with Background

In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 3)

x_FineStep, colors_all = Viz.make_fine_step(x_coor_all_clean, tau_smooth, step=2, color=color_blue, saturation=1, savgol_filter_level=(15, 3))
color_array = np.concatenate([np.expand_dims(x_FineStep, 1), colors_all], axis=1)

x_all_sample1 = np.copy(x_all)
y_all_sample1 = np.copy(y_all)
color_array_sample1 = np.copy(color_array)

fig, ax = plt.subplots(1, 1, figsize=(6.5, 2), layout='compressed')
Viz.draw_background_colors(ax, color_array_sample1)
ax.plot(x_all_sample1, y_all_sample1, color='k')
Viz.set_labels(ax, xlabel='Time (s)', ylabel='Intensity (a.u.)', title=None, xlim=(-2, 130))
plt.show()

In [ ]:
np.save('../Data/Plume_results/treated_213nm-fitting_results(sklearn).npy', np.concatenate([np.expand_dims(x_coor_all, 1), parameters_all], axis=1))
np.save('../Data/Plume_results/treated_213nm-bg_tau.npy', color_array_sample1)

### 2.2 Sample 2 - treated_81nm

#### 2.2.1 Fitting process

In [ ]:
path = 'D:/STO_STO-Data/RHEED/STO_STO_Berkeley/test7_gaussian_fit_parameters_all-04232023.h5'
D2_para = RHEED_parameter_dataset(path, camera_freq=500, sample_name='treated_81nm')

growth_list = ['growth_1', 'growth_2', 'growth_3', 'growth_4', 'growth_5']
D2_para.viz_RHEED_parameter_trend(growth_list, spot='spot_2', metric_list=['img_sum'], head_tail=(100,300), interval=200)

In [ ]:
growth_dict = {'growth_1':1, 'growth_2':1, 'growth_3':1, 'growth_4':3, 'growth_5':3}
x_all, y_all = D2_para.load_multiple_curves(growth_dict.keys(), spot, metric, x_start=0, interval=0)

parameters_all, x_coor_all, info = analyze_curves(D2_para, growth_dict, spot, metric, interval=0, fit_settings=fit_settings)
[xs_all, ys_all, ys_fit_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels_all, losses_all] = info

# define two color regime
x_y1 = x_coor_all[losses_all[:,0]>losses_all[:,1]]
x_y2 = x_coor_all[losses_all[:,0]<losses_all[:,1]]
color_array = Viz.two_color_array(x_coor_all, x_y1, x_y2, bgc1, bgc2, transparency=0.5)
color_array = np.concatenate([np.expand_dims(x_coor_all, 1), color_array], axis=1)
Viz.plot_fit_details(xs_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels=range(len(x_all)), 
                     save_name='S11-fitting_details_treated_81nm', printing=printing_plot)

### 2.2.2 Fitted parameters

In [ ]:
fig, axes = layout_fig(4, 1, figsize=(8, 10))
Viz.plot_curve(axes[0], x_all, y_all, plot_type='lineplot', xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 115), yaxis_style='sci')
Viz.plot_curve(axes[1], x_coor_all, parameters_all[:,0], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted a (a.u.)', xlim=(-2, 115))
Viz.plot_curve(axes[2], x_coor_all, parameters_all[:,1], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted b (a.u.)', xlim=(-2, 115))
Viz.plot_curve(axes[3], x_coor_all, parameters_all[:,2], plot_type='lineplot', xlabel='Time (s)', ylabel='Characteristic Time (s)', xlim=(-2, 115))
printing_plot.savefig(fig, 'S8-treated_81nm-a_b_tau')
plt.show()

#### 2.2.3 Remove Outliers

Because there is hard boundary when characteristic >= 1, so we can assume the characteristic data that >= 0.95 are outliers and remove from plot. And we also process the data with less noise to observe the trend.


In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 5)

x_coor_all_clean_sample2 = np.copy(x_coor_all_clean)
tau_clean_sample2 = np.copy(tau_clean)
tau_smooth_sample2 = np.copy(tau_smooth)

fig, ax = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.plot_curve(ax, x_coor_all_clean_sample2, tau_clean_sample2, curve_y_fit=tau_smooth_sample2, plot_type='lineplot', 
           plot_colors=['k', '#bc5090'], xlabel='Time (s)', ylabel='Characteristic Time (s)', 
           yaxis_style='linear', markersize=3, xlim=(-2, 115))

#### 2.2.4 Plot with Background

In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 3)

x_FineStep, colors_all = Viz.make_fine_step(x_coor_all_clean, tau_smooth, step=2, color=color_blue, saturation=1, savgol_filter_level=(15, 3))
color_array = np.concatenate([np.expand_dims(x_FineStep, 1), colors_all], axis=1)

x_all_sample2 = np.copy(x_all)
y_all_sample2 = np.copy(y_all)
color_array_sample2 = np.copy(color_array)

fig, ax = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax, color_array_sample2)
ax.plot(x_all_sample2, y_all_sample2, color='k')
Viz.set_labels(ax, xlabel='Time (s)', ylabel='Intensity (a.u.)', title=None, xlim=(-2, 115))
plt.show()

In [ ]:
np.save('../Data/Plume_results/treated_81nm-fitting_results(sklearn).npy', np.concatenate([np.expand_dims(x_coor_all, 1), parameters_all], axis=1))
np.save('../Data/Plume_results/treated_81nm-bg_tau.npy', color_array_sample2)

### 2.3 Sample 3 - untreated_162nm

#### 2.3.1 Fitting Process

In [ ]:
path = 'D:/STO_STO-Data/RHEED/STO_STO_Berkeley/test9_gaussian_fit_parameters_all-04232023.h5'
D3_para = RHEED_parameter_dataset(path, camera_freq=500, sample_name='untreated_162nm')

growth_list = ['growth_1', 'growth_2', 'growth_3', 'growth_4', 'growth_5']
D3_para.viz_RHEED_parameter_trend(growth_list, spot='spot_2', metric_list=['img_sum'], head_tail=(100,300), interval=200)

In [ ]:
growth_dict = {'growth_1':1, 'growth_2':1, 'growth_3':1, 'growth_4':3, 'growth_5':3}
x_all, y_all = D3_para.load_multiple_curves(growth_dict.keys(), spot, metric, x_start=0, interval=0)

parameters_all, x_coor_all, info = analyze_curves(D3_para, growth_dict, spot, metric, interval=0, fit_settings=fit_settings)
[xs_all, ys_all, ys_fit_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels_all, losses_all] = info

# define two color regime
x_y1 = x_coor_all[losses_all[:,0]>losses_all[:,1]]
x_y2 = x_coor_all[losses_all[:,0]<losses_all[:,1]]
color_array = Viz.two_color_array(x_coor_all, x_y1, x_y2, bgc1, bgc2, transparency=0.5)
color_array = np.concatenate([np.expand_dims(x_coor_all, 1), color_array], axis=1)
Viz.plot_fit_details(xs_all, ys_nor_all, ys_nor_fit_all, ys_nor_fit_failed_all, labels=range(len(x_all)), 
                     save_name='S12-fitting_details_untreated_162nm', printing=printing_plot)

#### 2.3.2 Fitted parameters

In [ ]:
fig, axes = layout_fig(4, 1, figsize=(8, 10))
Viz.plot_curve(axes[0], x_all, y_all, plot_type='lineplot', xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 125), yaxis_style='sci')
Viz.plot_curve(axes[1], x_coor_all, parameters_all[:,0], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted a (a.u.)', xlim=(-2, 125))
Viz.plot_curve(axes[2], x_coor_all, parameters_all[:,1], plot_type='lineplot', xlabel='Time (s)', ylabel='Fitted b (a.u.)', xlim=(-2, 125))
Viz.plot_curve(axes[3], x_coor_all, parameters_all[:,2], plot_type='lineplot', xlabel='Time (s)', ylabel='Characteristic Time (s)', xlim=(-2, 125))
printing_plot.savefig(fig, 'S9-untreated_162nm-a_b_tau')
plt.show()

#### 2.3.3 Remove outliers

Because there is hard boundary when characteristic >= 1, so we can assume the characteristic data that >= 0.95 are outliers and remove from plot. And we also process the data with less noise to observe the trend.


In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 5)

x_coor_all_clean_sample3 = np.copy(x_coor_all_clean)
tau_clean_sample3 = np.copy(tau_clean)
tau_smooth_sample3 = np.copy(tau_smooth)
fig, ax = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.plot_curve(ax, x_coor_all_clean_sample3, tau_clean_sample3, curve_y_fit=tau_smooth_sample3, plot_type='lineplot', 
           plot_colors=['k', '#bc5090'], xlabel='Time (s)', ylabel='Characteristic Time (s)', 
           yaxis_style='linear', markersize=3, xlim=(-2, 125))

#### 2.3.4 Plot with Background

In [ ]:
x_coor_all_clean, tau_clean = remove_outlier(x_coor_all, parameters_all[:,2], 0.95)
tau_smooth = smooth(tau_clean, 3)

x_FineStep, colors_all = Viz.make_fine_step(x_coor_all_clean, tau_smooth, step=2, color=color_blue, saturation=1, savgol_filter_level=(15, 3))
color_array = np.concatenate([np.expand_dims(x_FineStep, 1), colors_all], axis=1)

x_all_sample3 = np.copy(x_all)
y_all_sample3 = np.copy(y_all)
color_array_sample3 = np.copy(color_array)

fig, ax = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax, color_array_sample3)
ax.plot(x_all_sample3, y_all_sample3, color='k')
Viz.set_labels(ax, xlabel='Time (s)', ylabel='Intensity (a.u.)', title=None, xlim=(-2, 125))
plt.show()

In [ ]:
np.save('../Data/Plume_results/untreated_162nm-fitting_results(sklearn).npy', np.concatenate([np.expand_dims(x_coor_all, 1), parameters_all], axis=1))
np.save('../Data/Plume_results/untreated_162nm-bg_tau.npy', color_array_sample3)

## 3. Summary of Figures

### 3.1 Plots of Characteristic Times with RHEED intensity

#### 3.1.1 Sample 1 - treated_213nm

In [ ]:
x_all_sample1, y_all_sample1 = np.load('../Data/Plume_results/treated_213nm-x_all.npy'), np.load('../Data/Plume_results/treated_213nm-y_all.npy')
x_sklearn_sample1, tau_sklearn_sample1 = np.swapaxes(np.load('../Data/Plume_results/treated_213nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample1 = np.load('../Data/Plume_results/treated_213nm-bg_growth.npy')
bg_tau_sample1 = np.load('../Data/Plume_results/treated_213nm-bg_tau.npy')

y_all_sample1 = y_all_sample1[x_all_sample1<110]
x_all_sample1 = x_all_sample1[x_all_sample1<110]
tau_sklearn_sample1 = tau_sklearn_sample1[x_sklearn_sample1<110]
x_sklearn_sample1 = x_sklearn_sample1[x_sklearn_sample1<110]
bg_growth_sample1 = bg_growth_sample1[bg_growth_sample1[:, 0]<110]

x_sklearn_sample1, tau_clean_sample1 = remove_outlier(x_sklearn_sample1, tau_sklearn_sample1, 0.95)
tau_smooth_sample1 = smooth(tau_clean_sample1, 3)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
ax1.scatter(x_all_sample1, y_all_sample1, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 130), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample1, tau_clean_sample1, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample1, tau_smooth_sample1, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)

# colors.hex2color(seq_colors[0])
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
# ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
# ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'treated_213nm-no_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_tau_sample1)
ax1.scatter(x_all_sample1, y_all_sample1, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 130), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample1, tau_clean_sample1, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample1, tau_smooth_sample1, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'treated_213nm-tau_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_growth_sample1)
ax1.scatter(x_all_sample1, y_all_sample1, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 130), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample1, tau_clean_sample1, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample1, tau_smooth_sample1, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
plt.xlim(-2, 115)
printing_plot.savefig(fig, 'treated_213nm-growth_bg')
plt.show()

#### 3.1.2 Sample 2 - treated_81nm

In [ ]:
x_all_sample2, y_all_sample2 = np.load('../Data/Plume_results/treated_81nm-x_all.npy'), np.load('../Data/Plume_results/treated_81nm-y_all.npy')
x_sklearn_sample2, tau_sklearn_sample2 = np.swapaxes(np.load('../Data/Plume_results/treated_81nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample2 = np.load('../Data/Plume_results/treated_81nm-bg_growth.npy')
bg_tau_sample2 = np.load('../Data/Plume_results/treated_81nm-bg_tau.npy')

y_all_sample2 = y_all_sample2[x_all_sample2<110]
x_all_sample2 = x_all_sample2[x_all_sample2<110]
tau_sklearn_sample2 = tau_sklearn_sample2[x_sklearn_sample2<110]
x_sklearn_sample2 = x_sklearn_sample2[x_sklearn_sample2<110]
bg_growth_sample2 = bg_growth_sample2[bg_growth_sample2[:, 0]<110]

x_sklearn_sample2, tau_clean_sample2 = remove_outlier(x_sklearn_sample2, tau_sklearn_sample2, 0.95)
tau_smooth_sample2 = smooth(tau_clean_sample2, 3)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
ax1.scatter(x_all_sample2, y_all_sample2, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 115), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample2, tau_clean_sample2, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample2, tau_smooth_sample2, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'treated_81nm-no_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_tau_sample2)
ax1.scatter(x_all_sample2, y_all_sample2, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 115), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample2, tau_clean_sample2, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample2, tau_smooth_sample2, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'treated_81nm-tau_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_growth_sample2)
ax1.scatter(x_all_sample2, y_all_sample2, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 115), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample2, tau_clean_sample2, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample2, tau_smooth_sample2, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
plt.xlim(-2, 115)
printing_plot.savefig(fig, 'treated_81nm-growth_bg')
plt.show()

#### 3.1.3 Sample 3 - untreated_162nm

In [ ]:
x_all_sample3, y_all_sample3 = np.load('../Data/Plume_results/untreated_162nm-x_all.npy'), np.load('../Data/Plume_results/untreated_162nm-y_all.npy')
x_sklearn_sample3, tau_sklearn_sample3 = np.swapaxes(np.load('../Data/Plume_results/untreated_162nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample3 = np.load('../Data/Plume_results/untreated_162nm-bg_growth.npy')
bg_tau_sample3 = np.load('../Data/Plume_results/untreated_162nm-bg_tau.npy')

y_all_sample3 = y_all_sample3[x_all_sample3<110]
x_all_sample3 = x_all_sample3[x_all_sample3<110]
tau_sklearn_sample3 = tau_sklearn_sample3[x_sklearn_sample3<110]
x_sklearn_sample3 = x_sklearn_sample3[x_sklearn_sample3<110]
bg_growth_sample3 = bg_growth_sample3[bg_growth_sample3[:, 0]<110]

x_sklearn_sample3, tau_clean_sample3 = remove_outlier(x_sklearn_sample3, tau_sklearn_sample3, 0.95)
tau_smooth_sample3 = smooth(tau_clean_sample3, 3)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
ax1.scatter(x_all_sample3, y_all_sample3, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 200), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample3, tau_clean_sample3, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample3, tau_smooth_sample3, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'untreated_162nm-no_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_tau_sample3)
ax1.scatter(x_all_sample3, y_all_sample3, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 125), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample3, tau_clean_sample3, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample3, tau_smooth_sample3, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
printing_plot.savefig(fig, 'untreated_162nm-tau_bg')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(8, 2.5), layout='compressed')
Viz.draw_background_colors(ax1, bg_growth_sample3)
ax1.scatter(x_all_sample3, y_all_sample3, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 125), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample3, tau_clean_sample3, color=seq_colors[0], s=3)
# ax2.plot(x_sklearn_sample3, tau_smooth_sample3, color='#bc5090', markersize=3)
ax2.plot(x_sklearn_sample3, tau_smooth_sample3, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
plt.legend(['original', 'processed'], fontsize=8, loc="upper right", frameon=True)
plt.xlim(-2, 115)
printing_plot.savefig(fig, 'untreated_162nm-growth_bg')
plt.show()

### 3.2 Violin Plot

In [ ]:
x_sklearn_sample1, tau_sklearn_sample1

In [ ]:
x_all_sample1, y_all_sample1 = np.load('../Data/Plume_results/treated_213nm-x_all.npy'), np.load('../Data/Plume_results/treated_213nm-y_all.npy')
x_sklearn_sample1, tau_sklearn_sample1 = np.swapaxes(np.load('../Data/Plume_results/treated_213nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample1 = np.load('../Data/Plume_results/treated_213nm-bg_growth.npy')
bg_tau_sample1 = np.load('../Data/Plume_results/treated_213nm-bg_tau.npy')

# y_all_sample1 = y_all_sample1[x_all_sample1<110]
# x_all_sample1 = x_all_sample1[x_all_sample1<110]
# tau_sklearn_sample1 = tau_sklearn_sample1[x_sklearn_sample1<110]
# x_sklearn_sample1 = x_sklearn_sample1[x_sklearn_sample1<110]
# bg_growth_sample1 = bg_growth_sample1[bg_growth_sample1[:, 0]<110]

x_sklearn_sample1, tau_clean_sample1 = remove_outlier(x_sklearn_sample1, tau_sklearn_sample1, 0.95)
tau_smooth_sample1 = smooth(tau_clean_sample1, 3)

In [ ]:
x_all_sample2, y_all_sample2 = np.load('../Data/Plume_results/treated_81nm-x_all.npy'), np.load('../Data/Plume_results/treated_81nm-y_all.npy')
x_sklearn_sample2, tau_sklearn_sample2 = np.swapaxes(np.load('../Data/Plume_results/treated_81nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample2 = np.load('../Data/Plume_results/treated_81nm-bg_growth.npy')
bg_tau_sample2 = np.load('../Data/Plume_results/treated_81nm-bg_tau.npy')

# y_all_sample2 = y_all_sample2[x_all_sample2<110]
# x_all_sample2 = x_all_sample2[x_all_sample2<110]
# tau_sklearn_sample2 = tau_sklearn_sample2[x_sklearn_sample2<110]
# x_sklearn_sample2 = x_sklearn_sample2[x_sklearn_sample2<110]
# bg_growth_sample2 = bg_growth_sample2[bg_growth_sample2[:, 0]<110]

x_sklearn_sample2, tau_clean_sample2 = remove_outlier(x_sklearn_sample2, tau_sklearn_sample2, 0.9999)
tau_smooth_sample2 = smooth(tau_clean_sample2, 3)

In [ ]:
x_all_sample3, y_all_sample3 = np.load('../Data/Plume_results/untreated_162nm-x_all.npy'), np.load('../Data/Plume_results/untreated_162nm-y_all.npy')
x_sklearn_sample3, tau_sklearn_sample3 = np.swapaxes(np.load('../Data/Plume_results/untreated_162nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample3 = np.load('../Data/Plume_results/untreated_162nm-bg_growth.npy')
bg_tau_sample3 = np.load('../Data/Plume_results/untreated_162nm-bg_tau.npy')

# y_all_sample3 = y_all_sample3[x_all_sample3<110]
# x_all_sample3 = x_all_sample3[x_all_sample3<110]
# tau_sklearn_sample3 = tau_sklearn_sample3[x_sklearn_sample3<110]
# x_sklearn_sample3 = x_sklearn_sample3[x_sklearn_sample3<110]
# bg_growth_sample3 = bg_growth_sample3[bg_growth_sample3[:, 0]<110]

x_sklearn_sample3, tau_clean_sample3 = remove_outlier(x_sklearn_sample3, tau_sklearn_sample3, 0.95)
tau_smooth_sample3 = smooth(tau_clean_sample3, 3)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2.5), layout='compressed')
titles = ['Treated substrate\n(step width=213±88nm)',
          'Treated substrate\n(step width=81±44nm)',
          'Untreated substrate\n(step width=162±83μm)']

ax = sns.violinplot(data=[tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], 
                    palette=[color_blue, color_orange, color_purple], linewidth=0.8, inner=None)
Viz.set_labels(ax, ylabel='Characteristic Time (s)', ticks_both_sides=False, yaxis_style='linear')
Viz.label_violinplot(ax, [tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], label_type='median', text_pos='center')
ax.set_xticklabels(titles)
printing.savefig(fig, 'violinplot_plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2.5), layout='compressed')
titles = ['Treated substrate\n(step width=213±88nm)',
          'Treated substrate\n(step width=81±44nm)',
          'Untreated substrate\n(step width=162±83μm)']

ax = sns.violinplot(data=[tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], 
                    palette=[color_blue, color_orange, color_purple], linewidth=0.8, inner=None)
Viz.set_labels(ax, ylabel='Characteristic Time (s)', ticks_both_sides=False, yaxis_style='linear')
Viz.label_violinplot(ax, [tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], label_type='average', text_pos='center')
ax.set_xticklabels(titles)
printing.savefig(fig, 'violinplot_plot')
plt.show()

In [ ]:
# align the data with time
tau_clean_sample1 = tau_clean_sample1[x_sklearn_sample1<110]
x_sklearn_sample1 = x_sklearn_sample1[x_sklearn_sample1<110]

tau_clean_sample2 = tau_clean_sample2[x_sklearn_sample2<110]
x_sklearn_sample2 = x_sklearn_sample2[x_sklearn_sample2<110]

tau_clean_sample3 = tau_clean_sample3[x_sklearn_sample3<110]
x_sklearn_sample3 = x_sklearn_sample3[x_sklearn_sample3<110]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2.5), layout='compressed')
titles = ['Treated substrate\n(step width=213±88nm)',
          'Treated substrate\n(step width=81±44nm)',
          'Untreated substrate\n(step width=162±83μm)']

ax = sns.violinplot(data=[tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], 
                    palette=[color_blue, color_orange, color_purple], linewidth=0.8, inner=None)
Viz.set_labels(ax, ylabel='Characteristic Time (s)', ticks_both_sides=False, yaxis_style='linear')
Viz.label_violinplot(ax, [tau_clean_sample1, tau_clean_sample2, tau_clean_sample3], label_type='average', text_pos='center')
ax.set_xticklabels(titles)
printing.savefig(fig, 'violinplot_plot')
plt.show()

### 3.3 Summary Figure 

In [ ]:
align_time = True
normalize = True

seq_colors = ['#00429d','#2e59a8','#4771b2','#5d8abd','#73a2c6','#8abccf','#a5d5d8','#c5eddf','#ffffe0']
fig, axes = layout_fig(3, 1, figsize=(8, 8))
ax1, ax3, ax5 = axes[0], axes[1], axes[2]

x_all_sample1, y_all_sample1 = np.load('../Data/Plume_results/treated_213nm-x_all.npy'), np.load('../Data/Plume_results/treated_213nm-y_all.npy')
x_sklearn_sample1, tau_sklearn_sample1 = np.swapaxes(np.load('../Data/Plume_results/treated_213nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample1 = np.load('../Data/Plume_results/treated_213nm-bg_growth.npy')

x_all_sample2, y_all_sample2 = np.load('../Data/Plume_results/treated_81nm-x_all.npy'), np.load('../Data/Plume_results/treated_81nm-y_all.npy')
x_sklearn_sample2, tau_sklearn_sample2 = np.swapaxes(np.load('../Data/Plume_results/treated_81nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample2 = np.load('../Data/Plume_results/treated_81nm-bg_growth.npy')

x_all_sample3, y_all_sample3 = np.load('../Data/Plume_results/untreated_162nm-x_all.npy'), np.load('../Data/Plume_results/untreated_162nm-y_all.npy')
x_sklearn_sample3, tau_sklearn_sample3 = np.swapaxes(np.load('../Data/Plume_results/untreated_162nm-fitting_results(sklearn).npy'), 0, 1)[[0, -1]]
bg_growth_sample3 = np.load('../Data/Plume_results/untreated_162nm-bg_growth.npy')


if align_time:
    y_all_sample1 = y_all_sample1[x_all_sample1<110]
    x_all_sample1 = x_all_sample1[x_all_sample1<110]
    tau_sklearn_sample1 = tau_sklearn_sample1[x_sklearn_sample1<110]
    x_sklearn_sample1 = x_sklearn_sample1[x_sklearn_sample1<110]
    bg_growth_sample1 = bg_growth_sample1[bg_growth_sample1[:, 0]<110]
    
    y_all_sample2 = y_all_sample2[x_all_sample2<110]
    x_all_sample2 = x_all_sample2[x_all_sample2<110]
    tau_sklearn_sample2 = tau_sklearn_sample2[x_sklearn_sample2<110]
    x_sklearn_sample2 = x_sklearn_sample2[x_sklearn_sample2<110]
    bg_growth_sample2 = bg_growth_sample2[bg_growth_sample2[:, 0]<110]
    
    y_all_sample3 = y_all_sample3[x_all_sample3<110]
    x_all_sample3 = x_all_sample3[x_all_sample3<110]
    tau_sklearn_sample3 = tau_sklearn_sample3[x_sklearn_sample3<110]
    x_sklearn_sample3 = x_sklearn_sample3[x_sklearn_sample3<110]
    bg_growth_sample3 = bg_growth_sample3[bg_growth_sample3[:, 0]<110]


if normalize:
    y_all_sample1 = NormalizeData(y_all_sample1, lb=np.min(y_all_sample1), ub=np.max(y_all_sample1))
    y_all_sample2 = NormalizeData(y_all_sample2, lb=np.min(y_all_sample2), ub=np.max(y_all_sample2))
    y_all_sample3 = NormalizeData(y_all_sample3, lb=np.min(y_all_sample3), ub=np.max(y_all_sample3))

x_sklearn_sample1, tau_clean_sample1 = remove_outlier(x_sklearn_sample1, tau_sklearn_sample1, 0.95)
tau_smooth_sample1 = smooth(tau_clean_sample1, 3)

x_sklearn_sample2, tau_clean_sample2 = remove_outlier(x_sklearn_sample2, tau_sklearn_sample2, 0.95)
tau_smooth_sample2 = smooth(tau_clean_sample2, 3)

x_sklearn_sample3, tau_clean_sample3 = remove_outlier(x_sklearn_sample3, tau_sklearn_sample3, 0.95)
tau_smooth_sample3 = smooth(tau_clean_sample3, 3)


Viz.draw_background_colors(ax1, bg_growth_sample1)
ax1.scatter(x_all_sample1, y_all_sample1, color='k', s=1)
Viz.set_labels(ax1, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 130), ticks_both_sides=False)

ax2 = ax1.twinx()
ax2.scatter(x_sklearn_sample1, tau_clean_sample1, color=seq_colors[0], s=3)
ax2.plot(x_sklearn_sample1, tau_smooth_sample1, color='#bc5090', markersize=3)
Viz.set_labels(ax2, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax2.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax2.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
ax2.legend(['original', 'processed'], fontsize=8, loc="upper left", frameon=True, bbox_to_anchor=(0.04, 1))


Viz.draw_background_colors(ax3, bg_growth_sample2)
ax3.scatter(x_all_sample2, y_all_sample2, color='k', s=1)
Viz.set_labels(ax3, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 115), ticks_both_sides=False)

ax4 = ax3.twinx()
ax4.scatter(x_sklearn_sample2, tau_clean_sample2, color=seq_colors[0], s=3)
ax4.plot(x_sklearn_sample2, tau_smooth_sample2, color='#bc5090', markersize=3)
Viz.set_labels(ax4, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax4.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax4.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
ax4.legend(['original', 'processed'], fontsize=8, loc="upper left", frameon=True, bbox_to_anchor=(0.25, 1))    


Viz.draw_background_colors(ax5, bg_growth_sample3)
ax5.scatter(x_all_sample3, y_all_sample3, color='k', s=1)
Viz.set_labels(ax5, xlabel='Time (s)', ylabel='Intensity (a.u.)', xlim=(-2, 125), ticks_both_sides=False)

ax6 = ax5.twinx()
ax6.scatter(x_sklearn_sample3, tau_clean_sample3, color=seq_colors[0], s=3)
ax6.plot(x_sklearn_sample3, tau_smooth_sample3, color='#bc5090', markersize=3)
Viz.set_labels(ax6, ylabel='Characteristic Time (s)', yaxis_style='lineplot', ylim=(-0.05, 0.5), ticks_both_sides=False)
ax6.tick_params(axis="y", color='k', labelcolor=seq_colors[0])
ax6.set_ylabel('Characteristic Time (s)', color=seq_colors[0])
ax6.legend(['original', 'processed'], fontsize=8, loc="upper left", frameon=True, bbox_to_anchor=(0.07, 1)) 

if align_time:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.set_xlim(-2, 115)
        
printing_plot.savefig(fig, 'growth_bg-characterization_time')
plt.show()